#Ambiente

In [ ]:
# Install environment dependencies
%pip install unidecode nltk pandas numpy matplotlib seaborn scikit-learn yellowbrick setuptools plotly nbformat gensim==4.3.1 scipy==1.10.1

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import nltk
nltk.download('punkt')

# Datasets

## AmericanasBR

In [ ]:
#baixando os datasets
!curl https://www.inf.ufrgs.br/~viviane/DS/B2W-Reviews01_binario5000_TRAIN.csv > B2W-Reviews01_binario5000_TRAIN.csv

In [ ]:
df_train = pd.read_csv('B2W-Reviews01_binario5000_TRAIN.csv')

In [ ]:
classes = df_train.label.unique()
classes

# Entendendo as Embeddigns

A bilbioteca [Gensim](https://radimrehurek.com/gensim/models/word2vec.html) permite treinar e usar word embedings.

A versão da biblioteca a ser usada neste notebook é a 4.

Veja diferenças entre versão 3 e 4 neste [link](https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).

In [ ]:
from gensim import utils
import gensim.models

In [ ]:
gensim.__version__

## Treinar embeddings

A biblioteca Gensim permite que você treine as embeddings do seu corpus

https://radimrehurek.com/gensim/models/word2vec.html

In [ ]:
# classe para montar o dataset
class PreProcess:
    def __init__(self, docs):
            self.lista_text = docs
    def __iter__(self):
        for line in self.lista_text:
            # assume there's one document per line, tokens separated by whitespace:
            yield utils.simple_preprocess(line) # este método tokeniza e faz algum preprocessamento
            # https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html

Alguns parametros do [Word2vec](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)

*   `vector_size` – dimensionalidade dos vetores das palavras.
*   `window` – tamanho do contexto a considerar, por exemplo windows=5 irá considerar as 5 palavras à esquerda e as 5 palavras à direita da palavra atual como a janela de contexto. O modelo tentará então prever a palavra atual dado este contexto.
*   `min_count` – ignora palavras com frequência total menor do que min_count.
*   `sg` – o algoritmo de treinamento: 1 for skip-gram e diferente disto CBOW.





In [ ]:
sentences = PreProcess(df_train['text'].values)
# assim treina o modelo usando as configurações padrão e estas especificadas aqui
model = gensim.models.Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, epochs=20, sg=1)

Quando não se necessita mais do estado completo do modelo treinado (não precisa continuar treinando), `Gensim` permite separar os vetores treinados em [`KeyedVectors`](https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors) possibilitando salvar apenas os vetores e suas chaves (as palavras).



In [ ]:
# desta forma acessa somente as palavras e seus vetores
word_vectors = model.wv
word_vectors

In [ ]:
# total de palavras e as 10 primeiras
words = list(word_vectors.key_to_index)
print(f'O vocabulario contém {len(words)} palavras')
print(words[0:10])

In [ ]:
# verificando o id de uma palavra:
print('id de entrega:', word_vectors.key_to_index['entrega'])
print('palavra do id 4:', word_vectors.index_to_key[4])

In [ ]:
# ocorrências de uma palavra:
palavra = 'entrega'
palavra_cnt = word_vectors.get_vecattr(palavra, "count")
print(f'A palavra {palavra} ocorre {palavra_cnt} vezes no dataset')

Cada palavra única do corpus é representada por um vetor de tamanho `vector_size`, que corresponde ao número de dimnesões usado no treinamento.

In [ ]:
print(f"Embeddings da palavra produto com dimensão {word_vectors['produto'].shape}")
word_vectors['produto']

In [ ]:
# possuem representações diferentes:
print(word_vectors['agua'][0:5])
print(word_vectors['água'][0:5])

Salvando as embeddings treinadas:

In [ ]:
# salva o modelo em formato binario do gensim:
model.save("word2vec.model")

In [ ]:
# salva em formato texto somete as palavras e seus vetores de embeddings
word_vectors = model.wv
word_vectors.save_word2vec_format("word2vec.txt", binary= False)


## Lendo os vetores

In [ ]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('word2vec.txt', binary=False)

In [ ]:
word_vectors

In [ ]:
print('Total de palavras: ',len(word_vectors))
print('id da palavra água:', word_vectors.key_to_index['água'])
print(word_vectors['água'][0:10])

In [ ]:
#somente os vetores das embeddings:
vectors = word_vectors.vectors
vectors

In [ ]:
# acessando o vetor da palavra água
print(vectors[206][0:10])

## Visualizando embeddings

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

In [ ]:
%%time
tsne = TSNE(n_components=3, random_state=0)
projections = tsne.fit_transform(vectors, )

In [ ]:
dfP = pd.DataFrame(projections)
dfP['word'] = words
fig = px.scatter_3d(dfP, x=0, y=1, z=2,hover_data=['word'])
fig.update_traces(marker_size=3)
fig.show()

Também é possível usar o [Embeddings Projector](https://projector.tensorflow.org/) do Tensorflow.

## Usando embeddings já treinadas
Podemos usar word embeddings que já foram treinadas e disponibilizadas

Existem modelos disponíveis no Gensim:

In [ ]:
import gensim.downloader as api

In [ ]:
disponiveis = api.info()
disponiveis.keys()

In [ ]:
disponiveis['models'].keys()

In [ ]:
word_vectors = api.load("glove-wiki-gigaword-100") #128Mb

In [ ]:
#é uma lista de palavras e seus vetores de embeddinsg treinados por alguém e em algum algoritmo que normlmente é especificado na nomenclatura do arquivo juntamente com o número de dimensões
word_vectors

## Operações com embeddings

A similaridade entre vetores de embeddings é dada pelo cosseno. Quanto mais próximo de 1 mais similar

In [ ]:
similarity = word_vectors.similarity('woman', 'man')
similarity

In [ ]:
word_vectors.similarity('vehicle', 'car')

In [ ]:
similarity = word_vectors.similarity('woman', 'fruit')
similarity

In [ ]:
import numpy as np
from numpy.linalg import norm

In [ ]:
A = word_vectors['woman']
B = word_vectors['man']

In [ ]:
# lembrando que o cosseno é o produto escalar normalizado
cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)


queen = (king - man) + woman

In [ ]:
#testando o exemplo famoso do artigo do Mikolov sobre word2vec
result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

In [ ]:
result = word_vectors.most_similar(positive=['woman', 'programmer'], negative=['man'], topn=1)
print(result)

## Material suplementar - outras embeddings

Glove: http://github.com/stanfordnlp/glove


Word2vec treinado no detalhe no Keras:
https://www.tensorflow.org/tutorials/text/word2vec


Doc2Vec

https://cs.stanford.edu/~quocle/paragraph_vector.pdf

https://alvinntnu.github.io/python-notes/nlp/doc2vec.html

# Exercícios para entregar

Carregue as embeddings indicadas em português para os três exercicios.

In [ ]:
#baixando as embeddings do NILC de http://nilc.icmc.usp.br/embeddings
!curl http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s100.zip > cbow_s100.zip
!unzip -o cbow_s100.zip
nomearq = 'cbow_s100.txt'

In [ ]:
%%time
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(nomearq, binary=False)
print('Carregado: ',nomearq)

In [ ]:
# total de palavras e as 10 primeiras
words = list(word_vectors.key_to_index)
print(f'O vocabulario contém {len(words)}')

## Exercício 1

A polissemia ocorre quando uma mesma palavra possui mais de um significado. Um exemplo de polissemia é a palavra “manga”, que pode ser parte de vestimenta ou uma fruta.

a) Usando a função `most_similar` do Gensim, analise o resultado para a palavra "manga". Escolha outra palavra polissêmica que exista no vocabulário e verifique as palavras mais similares.

In [ ]:
# seu codigo aqui

b) O que você observa e qual a sua hipótese para explicar esse comportamento.

> Sua análise aqui

## Exercício 2

a) Faça o exercício para duas palavras:

Escolha uma palavra, um sinônimo e um antônimo da mesma.
Calcule a distância euclideana e a similaridade do cosseno entre a palavra e seu sinônimo e a palavra e seu antônimo.






In [ ]:
# seu codigo aqui

b) O que você observa e qual a sua hipótese para explicar esse comportamento.

> Sua análise aqui

## Exercício 3

a) Verifique as palavras mais similares em relação às palavras "enfermeiro" e "enfermeira".

In [ ]:
# seu codigo aqui

b) Você observa algum viés? Se sim, qual sua hipótese.


> Sua análise aqui

